In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
from sklearn import decomposition, manifold
from matplotlib import cm
import scipy.io

In [ ]:
def add_x_loc(x_loc, n_eachring = 32):
    """Input activity given location."""
    pref  = np.arange(0,2*np.pi,2*np.pi/n_eachring)
    dist = get_dist(x_loc-pref)  # periodic boundary
    dist /= np.pi/8
    return 0.8*np.exp(-dist**2/2)

def add_x_noise(x, seed=0,sigma_x = 0.01):
    rng = np.random.RandomState(seed)
    x += rng.randn(len(x1))*sigma_x
    return x

def get_dist(original_dist):
    '''Get the distance in periodic boundary conditions'''
    return np.minimum(abs(original_dist),2*np.pi-abs(original_dist))

n_loc = 128
n_stim_loc1, n_stim_loc2, repeat = stim_loc_shape = n_loc, n_loc, 1
stim_loc_size = np.prod(stim_loc_shape)
ind_stim_loc1, ind_stim_loc2, ind_repeat = np.unravel_index(range(stim_loc_size),stim_loc_shape)
stim1_locs = 2*np.pi*ind_stim_loc1/n_stim_loc1
stim2_locs = 2*np.pi*ind_stim_loc2/n_stim_loc2

In [ ]:
128*128/512

In [ ]:
stim1_locs[:5]

In [ ]:
def forward_prop(X, n_Z=256, activation=None, seed=0):
    rng = np.random.RandomState(seed)
    n_X = X.shape[1]
    W = rng.randn(n_Z, n_X) * 0.01
    Z = []
    for i in range(len(X)):
        Z.append(np.dot(W, X[i]))
    Z = np.array(Z)
    if activation == 'softplus':
        Z = tf.math.softplus(Z).numpy()
    elif activation == 'sigmoid':
        Z = tf.math.sigmoid(Z).numpy()
    return Z, W

In [ ]:
palette1 = cm.get_cmap('autumn',n_loc+15)
palette1 = [palette1(i)[:3] for i in range(n_loc)]
color1=np.array(palette1)[ind_stim_loc1]

palette2 = cm.get_cmap('summer',n_loc+15)
palette2 = [palette2(i)[:3] for i in range(n_loc)]
color2=np.array(palette2)[ind_stim_loc2]

palette1 = cm.get_cmap('autumn',n_loc+15)
palette1 = [palette1(i)[:3] for i in range(n_loc)]
color1=np.array(palette1)[ind_stim_loc1]

palette2 = cm.get_cmap('summer',n_loc+15)
palette2 = [palette2(i)[:3] for i in range(n_loc)]
color2=np.array(palette2)[ind_stim_loc2]

def plot_tuning_curves(Z,filename=''):    
    fig,ax = plt.subplots(10,1,figsize=(5,30))
    for i in range(10):
        neuron = i*int(Z.shape[1]/10)
        for loc1 in [j*10 for j in range(int(128/10))]:
            df = pd.DataFrame({'first_stim':ind_stim_loc1[ind_stim_loc1==loc1],'second_stim':ind_stim_loc2[ind_stim_loc1==loc1],'activity':Z[:,neuron][ind_stim_loc1==loc1]})
            x = df['second_stim']
            y = df['activity']
            ax[i].scatter(x,y,s=1, color=palette1[loc1], label=loc1)
        ax[i].set_ylabel('%dth neuron'%neuron)
        ax[i].legend()
        plt.xlabel('Stim 2 direction (deg)')
    if filename != '': 
        fig.savefig('stim1'+filename)
    plt.show()
    plt.close()
        
    fig,ax = plt.subplots(10,1,figsize=(5,30))
    for i in range(10):
        neuron = i*int(Z.shape[1]/10)
        for loc2 in [j*10 for j in range(int(128/10))]:
            df = pd.DataFrame({'first_stim':ind_stim_loc1[ind_stim_loc2==loc2],'second_stim':ind_stim_loc2[ind_stim_loc2==loc2],'activity':Z[:,neuron][ind_stim_loc2==loc2]})
            x = df['first_stim']
            y = df['activity']
            ax[i].scatter(x,y,s=1, color=palette2[loc2], label=loc2)
        ax[i].set_ylabel('%dth neuron'%neuron)
        ax[i].legend()
        plt.xlabel('Stim 1 direction (deg)')
    if filename != '':
        fig.savefig('stim2'+filename)
    plt.show()
    plt.close()
    
def fit_isomap(data_to_use, n_neighbors = 15, target_dim = 3):
    iso_instance = manifold.Isomap(n_neighbors = n_neighbors, n_components = target_dim)
    proj = iso_instance.fit_transform(data_to_use)
    return proj

def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

def plot_isomap(data_plot, color, annotate=False):
    fig = plt.figure(figsize=(16,16),dpi=200)
    ax = fig.add_subplot(111, projection='3d')

    if annotate:
        ax.scatter(data_plot[:,0], data_plot[:,1], data_plot[:,2], 
            s=5, alpha=1, edgecolor='face',c=color)
        label = 0
        for xyz in zip(data_plot[:,0], data_plot[:,1], data_plot[:,2]):
            x, y, z = xyz
            ax.text(x, y, z, '%s' % (label), size=5, zorder=1, color='k')
            label += 1
    else:
        ax.scatter(data_plot[:,0], data_plot[:,1], data_plot[:,2], 
            s=20, alpha=1, edgecolor='face',c=color)
    ax.grid(False)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')
    return fig, ax

def plot_single_distractor_or_target(palette, xlim, ylim, zlim, label_plot, proj_plot, annotate=False, filename=''):

    color=np.array(palette)[label_plot]

    # h0_longest,h1_longest,h2_longest = run_ripser(proj_plot,figure_dir+'ripser'+figure_subscript)
    fig, ax = plot_isomap(data_plot=proj_plot, color=color, annotate=annotate)
    plt.setp(ax, xlim=xlim, ylim=ylim, zlim=zlim)
    fig.tight_layout()
    if filename is not None:
        fig.savefig(filename)
    plt.show()
    plt.close(fig) 


def plot_all_isomap_figures(proj,filename=''):
    fig,ax = plot_isomap(data_plot=proj, color=color1)
    set_axes_equal(ax)
    fig.tight_layout()
    if filename is not None:
        fig.savefig('target_isomap_'+filename)
    plt.show()
    plt.close(fig)

    fig,ax = plot_isomap(data_plot=proj, color=color2)
    set_axes_equal(ax)
    fig.tight_layout()
    if filename is not None:
        fig.savefig('distractor_isomap_'+filename)
    plt.show()
    plt.close(fig)

    xlim=fig.gca().get_xlim()
    ylim=fig.gca().get_ylim()
    zlim=fig.gca().get_zlim()

    num=0
    indices = ind_stim_loc1==num
    label_plot = ind_stim_loc2[indices]
    proj_plot = proj[indices,:]
    plot_single_distractor_or_target(palette = palette2, xlim = xlim, ylim = ylim, zlim = zlim, label_plot=label_plot, proj_plot = proj_plot, filename = 'single_target_'+filename)

    num=0
    indices = ind_stim_loc2==num
    label_plot = ind_stim_loc1[indices]
    proj_plot = proj[indices,:]
    plot_single_distractor_or_target(palette = palette1, xlim = xlim, ylim = ylim, zlim = zlim, label_plot=label_plot, proj_plot = proj_plot, filename = 'single_distractor_'+filename)

In [ ]:
def get_delay_bins(delay):
    dt=20
    stim1_ons = int(500/dt)
    stim1_offs = stim1_ons + int(300/dt)
    stim2_ons =stim1_offs + int(1000/dt)
    stim2_offs = stim2_ons + int(300/dt)
    fix_offs  = stim2_offs + int(1000/dt)

    baseline = (0,stim1_ons)

    if delay == 1:
            delay_bins = (stim2_ons - int(500/dt),stim2_ons)

    elif delay == 2:
            delay_bins = (fix_offs - int(500/dt),fix_offs)

    return delay_bins

In [ ]:
X_n = []
for i in range(stim_loc_size):
    for seed in range(10):
        x1 = add_x_loc(stim1_locs[i])
        x2 = add_x_loc(stim2_locs[i])
        x1 = add_x_noise(x1, seed=seed)
        x2 = add_x_noise(x2, seed=seed)
        X_n.append(np.append(x1,x2))
X_n = np.array(X_n)

In [ ]:
X = []
for i in range(stim_loc_size):
    x1 = add_x_loc(stim1_locs[i])
    x2 = add_x_loc(stim2_locs[i])
    X.append(np.append(x1,x2))
X = np.array(X)

### ANOVA test for simple NN models (no non-linear activation)

In [ ]:
Z = forward_prop(X_n)
neuron = 0
df = pd.DataFrame({'first_stim':np.array([[i]*10 for i in ind_stim_loc1]).flatten(),'second_stim':np.array([[i]*10 for i in ind_stim_loc2]).flatten(),'activity':Z[:,neuron]})
df

In [ ]:
aov = pg.anova(dv='activity', between=['first_stim', 'second_stim'], data=df,
             detailed=True)

print(aov)

### ANOVA test for simple NN models (softmax activation)

In [ ]:
Z = forward_prop(X_n, activation='softplus')
neuron = 0
df = pd.DataFrame({'first_stim':np.array([[i]*10 for i in ind_stim_loc1]).flatten(),'second_stim':np.array([[i]*10 for i in ind_stim_loc2]).flatten(),'activity':Z[:,neuron]})
df

In [ ]:
aov = pg.anova(dv='activity', between=['first_stim', 'second_stim'], data=df,
             detailed=True)

print(aov)

### ANOVA test for simple NN Model with additional layer(s)

In [ ]:
Z = forward_prop(X_n, activation='softplus', seed=0)
Z = forward_prop(Z, activation='softplus', seed=1)
neuron = 0
df = pd.DataFrame({'first_stim':np.array([[i]*10 for i in ind_stim_loc1]).flatten(),'second_stim':np.array([[i]*10 for i in ind_stim_loc2]).flatten(),'activity':Z[:,neuron]})
df

In [ ]:
aov = pg.anova(dv='activity', between=['first_stim', 'second_stim'], data=df,
             detailed=True)

print(aov)

### ANOVA test for simple RNN model

In [ ]:
Z = forward_prop(X_n[:,:32], activation='softplus', seed=0)
Z = forward_prop(Z, seed=1) + forward_prop(X_n[:,32:], seed=0)
Z = tf.math.softplus(Z).numpy()
neuron = 0
df = pd.DataFrame({'first_stim':np.array([[i]*10 for i in ind_stim_loc1]).flatten(),'second_stim':np.array([[i]*10 for i in ind_stim_loc2]).flatten(),'activity':Z[:,neuron]})
df

In [ ]:
aov = pg.anova(dv='activity', between=['first_stim', 'second_stim'], data=df,
             detailed=True)

print(aov)

### ANOVA test for simple NN Model with additional layer(s)

In [ ]:
Z = forward_prop(X_n, activation='softplus', seed=0)
Z = forward_prop(Z, activation='softplus', seed=1)
neuron = 0
df = pd.DataFrame({'first_stim':np.array([[i]*10 for i in ind_stim_loc1]).flatten(),'second_stim':np.array([[i]*10 for i in ind_stim_loc2]).flatten(),'activity':Z[:,neuron]})
df

In [ ]:
aov = pg.anova(dv='activity', between=['first_stim', 'second_stim'], data=df,
             detailed=True)

print(aov)

### ANOVA test for simple NN Model with additional layer(s)

In [ ]:
Z = forward_prop(X_n, activation='softplus', seed=0)
Z = forward_prop(Z, activation='softplus', seed=1)
neuron = 0
df = pd.DataFrame({'first_stim':np.array([[i]*10 for i in ind_stim_loc1]).flatten(),'second_stim':np.array([[i]*10 for i in ind_stim_loc2]).flatten(),'activity':Z[:,neuron]})
df

In [ ]:
aov = pg.anova(dv='activity', between=['first_stim', 'second_stim'], data=df,
             detailed=True)

print(aov)

### ANOVA test for simple RNN model

In [ ]:
Z = forward_prop(X[:,:32], activation='softplus', seed=0)
Z = forward_prop(Z, seed=1) + forward_prop(X[:,32:], seed=0)
Z = tf.math.softplus(Z).numpy()
Z = forward_prop(X_n, activation='softplus')
neuron = 0
df = pd.DataFrame({'first_stim':np.array([[i]*10 for i in ind_stim_loc1]).flatten(),'second_stim':np.array([[i]*10 for i in ind_stim_loc2]).flatten(),'activity':Z[:,neuron]})
df

In [ ]:
aov = pg.anova(dv='activity', between=['first_stim', 'second_stim'], data=df,
             detailed=True)

print(aov)

### ANOVA test for trained RNN model

### Input layer

In [ ]:
proj = fit_isomap(data_to_use=X)
plot_all_isomap_figures(proj)

In [ ]:
plot_tuning_curves(X,'Xtuningcurve.png')

### Simple NN model

In [ ]:
Z,_ = forward_prop(X, activation='softplus', seed=0)
plot_tuning_curves(Z,'simpleNNtuningcurve.png')

In [ ]:
Z,_ = forward_prop(X, activation='softplus', seed=0)

plot_tuning_curves(Z,'simpleNNtuningcurve_softplus.png')

In [ ]:
proj = fit_isomap(data_to_use=Z)
plot_all_isomap_figures(proj)

### Simple NN Model with additional layer(s)

In [ ]:
Z,_ = forward_prop(X, activation='softplus', seed=0)
Z,_ = forward_prop(Z, activation='softplus', seed=1)

plot_tuning_curves(Z,'simpleNNtuningcurve_2softplus.png')

In [ ]:
proj = fit_isomap(data_to_use=Z)
plot_all_isomap_figures(proj)

In [ ]:
for i in range(2,10):
    Z,_ = forward_prop(Z, activation='softplus', seed=i)
plot_tuning_curves(Z,'simpleNNtuningcurve_10softplus.png')

In [ ]:
proj = fit_isomap(data_to_use=Z)
plot_all_isomap_figures(proj)

### Simple RNN

In [ ]:
Z,_ = forward_prop(X[:,:32], activation='softplus', seed=0)
Z,_ = forward_prop(Z, seed=1)
Z += forward_prop(X[:,32:], seed=0)[0]
Z = tf.math.softplus(Z).numpy()

In [ ]:
plot_tuning_curves(Z,'simpleRNNtuningcurve')

In [ ]:
proj = fit_isomap(data_to_use=Z)
plot_all_isomap_figures(proj,'simple_RNN.png')

In [ ]:
def plot_2d_isomap(data_plot, color, annotate=False):
        fig = plt.figure(figsize=(16,16),dpi=200)
        ax = fig.add_subplot(111)
        if annotate:
            ax.scatter(data_plot[:,0], data_plot[:,1], s=5, alpha=1, edgecolor='face',c=color)
            label = 0
            for xy in zip(data_plot[:,0], data_plot[:,1]):
                x, y = xy
                ax.text(x, y, '%s' % (label), size=15, zorder=1, color='k')
                label += 1
        else:
            ax.scatter(data_plot[:,0], data_plot[:,1], s=20, alpha=1, edgecolor='face',c=color)

        ax.grid(False)
        return fig, ax
    
num=0
target_dim = 2
indices = ind_stim_loc1==num
label_plot = ind_stim_loc2[indices]
data_to_use = Z[indices,:]

palette2 = cm.get_cmap('summer',n_loc+15)
palette2 = [palette2(i)[:3] for i in range(n_loc)]
color2=np.array(palette2)[label_plot]

iso_instance = manifold.Isomap(n_neighbors = n_neighbors, n_components = target_dim)
proj = iso_instance.fit_transform(data_to_use)

fig,ax = plot_2d_isomap(data_plot=proj, color=color2)
fig.tight_layout()
fig.savefig('simple_RNN_single_target_isomap_2d.png')
plt.show()
plt.close()



indices = ind_stim_loc2==num
label_plot = ind_stim_loc1[indices]
data_to_use = Z[indices,:]

palette2 = cm.get_cmap('summer',n_loc+15)
palette2 = [palette2(i)[:3] for i in range(n_loc)]
color1=np.array(palette1)[label_plot]

iso_instance = manifold.Isomap(n_neighbors = n_neighbors, n_components = target_dim)
proj = iso_instance.fit_transform(data_to_use)

fig,ax = plot_2d_isomap(data_plot=proj, color=color1)
fig.tight_layout()
fig.savefig('simple_RNN_single_distractor_isomap_2d.png')
plt.show()
plt.close()


In [ ]:
Z,_ = forward_prop(X[:,:32], activation='softplus', seed=0)
Z,param = forward_prop(Z, seed=1)
Z += forward_prop(X[:,32:], seed=0)[0]
Z = tf.math.softplus(Z).numpy()

In [ ]:
import seaborn as sns

def plot_weight_selectivity_matrix(target,Z,n_loc):
    mean_delay_FR = np.array([[np.mean(Z[target==i,j]) for i in range(n_loc)] for j in range(Z.shape[1])])
    df = pd.DataFrame(mean_delay_FR)
    label = np.argmax(mean_delay_FR,1)
    selectivity_label = np.array(label) 
    ind = np.argsort(selectivity_label)
    sorted_weight_matrix = param[:,ind]
    sorted_weight_matrix = sorted_weight_matrix[ind,:]
    
    #initialize matrix
    block_dist = np.zeros([n_loc,n_loc])

    for i in range(n_loc):
        for j in range(n_loc):
            from_neurons = np.where(label == i)[0]
            to_neurons = np.where(label == j)[0]
            block_dist[i,j] = np.nanmean(sorted_weight_matrix[from_neurons,:][:,to_neurons])

    highest = np.nanmax(np.abs(block_dist))
    fig = plt.figure()
    sns.heatmap(block_dist,cmap='bwr',vmin=-highest, vmax=highest)

    plt.show()

In [ ]:
plot_weight_selectivity_matrix(ind_stim_loc1,Z,n_loc=128)

In [ ]:
plot_weight_selectivity_matrix(ind_stim_loc2,Z,n_loc=128)

### trained RNN

In [ ]:
fpath = "/Volumes/Seagate Backup Plus Drive/fyp/run_0_unrestricted_128loc_noise_free"
isomap_file = "firing_rate_128testloc.mat"
n_loc = 128
delay = 2

model_dir = fpath
data = scipy.io.loadmat(model_dir + '/%s'%isomap_file) #data = dict(firing_rate [time bins,120*106 trials,256 neurons], [120*106 target locations], [120*106 distractor locations])

h = data['firing_rate']


target_label = data['target'][0][:]
distractor_label = data['distractor'][0][:]

palette1 = cm.get_cmap('autumn',n_loc+15)
palette1 = [palette1(i)[:3] for i in range(n_loc)]
palette2 = cm.get_cmap('summer',n_loc+15)
palette2 = [palette2(i)[:3] for i in range(n_loc)]

#get delay 1 or delay 2 time bins
delay_bins = get_delay_bins(delay=delay)
#extract mean firing rates for delay bins
df = pd.DataFrame(np.mean(h[delay_bins[0]:delay_bins[1], :, :], axis=0))
df = np.array(df)

In [ ]:
plot_weight_selectivity_matrix(distractor_label,df,n_loc=8)

In [ ]:
plot_weight_selectivity_matrix(target_label,df,n_loc=8)

In [ ]:
fpath = "/Volumes/Seagate Backup Plus Drive/fyp/run_0_unrestricted_128loc_noise_free"
isomap_file = "firing_rate_isomap_128testloc.mat"
n_loc = 128
delay = 2

model_dir = fpath
data = scipy.io.loadmat(model_dir + '/%s'%isomap_file) #data = dict(firing_rate [time bins,120*106 trials,256 neurons], [120*106 target locations], [120*106 distractor locations])

h = data['firing_rate']


target_label = data['target'][0][:]
distractor_label = data['distractor'][0][:]

palette1 = cm.get_cmap('autumn',n_loc+15)
palette1 = [palette1(i)[:3] for i in range(n_loc)]
palette2 = cm.get_cmap('summer',n_loc+15)
palette2 = [palette2(i)[:3] for i in range(n_loc)]

#get delay 1 or delay 2 time bins
delay_bins = get_delay_bins(delay=delay)
#extract mean firing rates for delay bins
df = pd.DataFrame(np.mean(h[delay_bins[0]:delay_bins[1], :, :], axis=0))
df = np.array(df)

In [ ]:
plot_weight_selectivity_matrix(ind_stim_loc1,df)
plot_weight_selectivity_matrix(ind_stim_loc2,df)

In [ ]:
df.shape

In [ ]:
#get delay 1 or delay 2 time bins
delay_bins = get_delay_bins(delay=1)
#extract mean firing rates for delay bins
df1 = pd.DataFrame(h[delay_bins[1], :, :])
df1 = np.array(df1)

In [ ]:
plot_tuning_curves(df1,'trainedRNNtuningcurve_delay1')

In [ ]:
#get delay 1 or delay 2 time bins
delay_bins = get_delay_bins(delay=2)
#extract mean firing rates for delay bins
df2 = pd.DataFrame(h[delay_bins[1], :, :])
df2 = np.array(df2)

In [ ]:
plot_tuning_curves(df2,'trainedRNNtuningcurve_delay2')

In [ ]:
df.shape

In [ ]:
aov = pg.anova(dv='activity', between=['first_stim', 'second_stim'], data=df,
             detailed=True)

print(aov)

In [ ]:
df_anova = pd.DataFrame({'first_stim':ind_stim_loc1,'second_stim':ind_stim_loc2,'activity':df[:,0]})
df_anova['first_stim']=(df_anova['first_stim']/2).astype('int').astype('object')
df_anova['second_stim']=(df_anova['second_stim']/2).astype('int').astype('object')

In [ ]:
df_anova
#generate extra points in hpc

In [ ]:
aov = pg.anova(dv='activity', between=['first_stim', 'second_stim'], data=df_anova,
             detailed=True)

print(aov)

In [ ]:
proj = fit_isomap(data_to_use=df)
plot_all_isomap_figures(proj)

In [ ]:
noise = np.zeros(df.shape)
noise[ind_stim_loc1 > 120] += np.random.randn(noise[ind_stim_loc1 > 120].shape[0],noise[ind_stim_loc1 > 120].shape[1]) * 0.1
df1 = df + noise

In [ ]:
proj = fit_isomap(data_to_use=df1)
plot_all_isomap_figures(proj,'trainedRNN_noise0p1')

In [ ]:
plot_all_isomap_figures(proj,'trainedRNN_noise0p1')

In [ ]:
noise = np.zeros(df.shape)
noise[ind_stim_loc1 > 120] += np.random.randn(noise[ind_stim_loc1 > 120].shape[0],noise[ind_stim_loc1 > 120].shape[1]) * 1
df2 = df + noise

In [ ]:
proj = fit_isomap(data_to_use=df2)
plot_all_isomap_figures(proj,'trainedRNN_noise1')

In [ ]:
df = pd.DataFrame({'first_stim':np.array([[i]*10 for i in ind_stim_loc1]).flatten(),'second_stim':np.array([[i]*10 for i in ind_stim_loc2]).flatten(),'activity':Z[:,neuron]})

aov = pg.anova(dv='activity', between=['first_stim', 'second_stim'], data=df,
             detailed=True)

print(aov)